# Lab 08 - Soham Deshkhaire
## Regression Modeling on Employment Data

In [ ]:
# 1. Setup & Initialization
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
pio.renderers.default = "notebook"
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

In [1]:
# 2. Load Dataset
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("../../data/lightcast_job_postings.csv")
df = df.dropna(subset=["SALARY", "MAX_YEARS_EXPERIENCE", "MIN_YEARS_EXPERIENCE", "REMOTE_TYPE_NAME"])
df.show(5)

NameError: name 'spark' is not defined

In [ ]:
# 3. Feature Engineering
data = df.select("SALARY", "MAX_YEARS_EXPERIENCE", "MIN_YEARS_EXPERIENCE", "REMOTE_TYPE_NAME")
indexer = StringIndexer(inputCol="REMOTE_TYPE_NAME", outputCol="remote_index")
data = indexer.fit(data).transform(data)
encoder = OneHotEncoder(inputCol="remote_index", outputCol="remote_ohe")
data = encoder.fit(data).transform(data)
assembler = VectorAssembler(inputCols=["MAX_YEARS_EXPERIENCE", "MIN_YEARS_EXPERIENCE", "remote_ohe"], outputCol="features")
final_data = assembler.transform(data).select("SALARY", "features")

In [2]:
# 4. Train-Test Split
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

NameError: name 'final_data' is not defined

In [3]:
# 5. Linear Regression Model
lr = LinearRegression(labelCol="SALARY", featuresCol="features")
model = lr.fit(train_data)
test_results = model.evaluate(test_data)
print("RMSE:", test_results.rootMeanSquaredError)
print("R2:", test_results.r2)

NameError: name 'LinearRegression' is not defined

In [4]:
# 6. Diagnostic Plots
predictions = model.transform(test_data).select("prediction", "SALARY").toPandas()
predictions["residuals"] = predictions["SALARY"] - predictions["prediction"]

plt.figure(figsize=(6,6))
sns.scatterplot(x="SALARY", y="prediction", data=predictions)
plt.plot([predictions.SALARY.min(), predictions.SALARY.max()], [predictions.SALARY.min(), predictions.SALARY.max()], 'r--')
plt.title("Predicted vs Actual Salary")
plt.xlabel("Actual Salary")
plt.ylabel("Predicted Salary")
plt.grid(True)
plt.savefig("_output/pred_vs_actual.png")
plt.show()

NameError: name 'model' is not defined

In [5]:
# Residuals vs Predicted
plt.figure(figsize=(6,6))
sns.scatterplot(x="prediction", y="residuals", data=predictions)
plt.axhline(0, linestyle='--', color='red')
plt.title("Residuals vs Predicted")
plt.xlabel("Predicted Salary")
plt.ylabel("Residuals")
plt.grid(True)
plt.savefig("_output/residuals_vs_predicted.png")
plt.show()

NameError: name 'plt' is not defined

In [6]:
# 7. Summary
print("Intercept:", model.intercept)
print("Coefficients:", model.coefficients)

summary = model.summary
summary_df = pd.DataFrame({
    "Estimate": summary.coefficients,
    "Std Error": summary.coefficientStandardErrors,
    "t-Value": summary.tValues,
    "P-Value": summary.pValues
})
summary_df.head()

NameError: name 'model' is not defined